# Fine-Tuning Phi-3 for Pyomo Code Generation

This notebook documents the process of fine-tuning the `microsoft/Phi-3-mini-128k-instruct` model with Quantized LoRA (QLoRA) to improve its capability in generating Pyomo code based on user inputs and existing code contexts. The dataset was generated using GPT-4o and includes 72 training examples, 8 evaluation examples, and 20 test examples.

The training process utilizes QLoRA for efficient memory usage and the final model is saved and evaluated based on metrics such as faithfulness, answer relevancy, and code quality.

The project structure includes:
- `datasets/test`: Test dataset
- `datasets/train`: Training and evaluation datasets
- `checkpoints`: Fine-tuned model checkpoints
- `merged_model`: PEFT model merged with the original model

Evaluation shows significant improvement in faithfulness and answer similarity metrics after fine-tuning.


## Config

In [1]:
import os
import warnings

import nest_asyncio
from huggingface_hub import HfApi

nest_asyncio.apply()

# ROCm for GFX_1030
os.environ["HSA_OVERRIDE_GFX_VERSION"] = "10.3.0"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

warnings.filterwarnings("ignore")

api = HfApi()
hf_username = api.whoami()["name"]

# Debugging with unknown package error
# import logging

# # Set the logging level to DEBUG
# logging.basicConfig(
#     level=logging.DEBUG, format="%(asctime)s - %(levelname)s - %(message)s"
# )

In [2]:
import os

import bitsandbytes as bnb
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import transformers
from peft import LoraConfig
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

amdgpu.ids: No such file or directory
amdgpu.ids: No such file or directory


In [3]:
print(f"pytorch version {torch.__version__}")
print(f"bitsandbytes version {bnb.__version__}")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"working on {device}")

pytorch version 2.3.1+rocm6.0
bitsandbytes version 0.43.2.dev
working on cuda:0


## Preview

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "wickes1/Phi-3-mini-128k-instruct-pyomo"
model = AutoModelForCausalLM.from_pretrained(
    model_name, trust_remote_code=True, device_map=device
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)


`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from transformers import TextStreamer

input_text = "Add a constraint to ensure variable Q is at least 10."
context_text = "from pyomo.environ import *\n\nmodel = ConcreteModel()\nmodel.Q = Var(domain=NonNegativeReals)\nmodel.obj = Objective(expr=2 * model.Q, sense=maximize)\n\n# Define constraints\nmodel.constraint1 = Constraint(expr=model.Q <= 8)\n\n# Solve the model\nsolver = SolverFactory('glpk')\nresult = solver.solve(model)"
prompt = "<|system|>\nYou are a highly knowledgeable AI assistant specializing in Python programming and Pyomo models. Based on the user input question and the provided code snippet, your task is to generate the complete Python code needed to address the user's query.\n\nInstructions:\n1. Review the user input question carefully.\n2. Analyze the provided code snippet within the context given.\n3. Generate the full Python code required to answer the user's question, ensuring the code is functional and correctly formatted.\n\nNotes:\n- Ensure that the generated code is properly indented and follows Python best practices.\n- Include necessary import statements and any relevant comments to enhance code readability.\n- Verify that the code aligns with the user's specific requirements as described in the input and context.\n<|end|>\n<|user|>\nInput:\n{input}\n\nContext:\n{context}\n\nComplete Python Code:\n<|end|>\n<|assistant|>"

# Tokenize input
model_input = tokenizer(
    prompt.format(input=input_text, context=context_text), return_tensors="pt"
).to(device)

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
_ = model.generate(
    model_input.input_ids,
    streamer=text_streamer,
    max_new_tokens=512,
)

from pyomo.environ import *

model = ConcreteModel()
model.Q = Var(domain=NonNegativeReals)
model.obj = Objective(expr=2 * model.Q, sense=maximize)

# Define constraints
model.constraint1 = Constraint(expr=model.Q <= 8)
model.constraint2 = Constraint(expr=model.Q >= 10)

# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)
<|end|>


## Prompt

In [4]:
#  Prompt generation functions


def generate_prompt_str(data_point):
    prompt = f"""
<|system|>
You are a highly knowledgeable AI assistant specializing in Python programming and Pyomo models. Based on the user input question and the provided code snippet, your task is to generate the complete Python code needed to address the user's query.

Instructions:
1. Review the user input question carefully.
2. Analyze the provided code snippet within the context given.
3. Generate the full Python code required to answer the user's question, ensuring the code is functional and correctly formatted.

Notes:
- Ensure that the generated code is properly indented and follows Python best practices.
- Include necessary import statements and any relevant comments to enhance code readability.
- Verify that the code aligns with the user's specific requirements as described in the input and context.
<|end|>
<|user|>
Input:
{data_point["input"]}

Context:
{data_point["context"]}

Complete Python Code:
<|end|>
<|assistant|>{data_point["output"]}<|endoftext|>
""".strip()
    return {"text": prompt}


def generate_test_prompt_str(data_point):
    prompt = f"""
<|system|>
You are a highly knowledgeable AI assistant specializing in Python programming and Pyomo models. Based on the user input question and the provided code snippet, your task is to generate the complete Python code needed to address the user's query.

Instructions:
1. Review the user input question carefully.
2. Analyze the provided code snippet within the context given.
3. Generate the full Python code required to answer the user's question, ensuring the code is functional and correctly formatted.

Notes:
- Ensure that the generated code is properly indented and follows Python best practices.
- Include necessary import statements and any relevant comments to enhance code readability.
- Verify that the code aligns with the user's specific requirements as described in the input and context.
<|end|>
<|user|>
Input:
{data_point["input"]}

Context:
{data_point["context"]}

Complete Python Code:
<|end|>
<|assistant|>
""".strip()
    return {"text": prompt}

## Dataset

In [5]:
# Local dataset

# from datasets import load_dataset

# # Load dataset
# data_files = {"train": "./datasets/train/*.csv", "test": "./datasets/test/*.csv"}
# dataset = load_dataset("csv", data_files=data_files)
# train_test_split = dataset["train"].train_test_split(test_size=0.1, seed=42)

# train_dataset = train_test_split["train"]
# eval_dataset = train_test_split["test"]
# test_dataset = dataset["test"]

# # Apply the prompt generation functions using map
# train_dataset = train_dataset.map(generate_prompt_str)
# test_dataset = test_dataset.map(generate_test_prompt_str)
# eval_dataset = eval_dataset.map(generate_prompt_str)

# # Verify
# print(train_dataset)
# print(eval_dataset)
# print(test_dataset)

# dataset.push_to_hub(f"{hf_user}/pyomo-100")

In [6]:
# Remote Dataset

from datasets import load_dataset

dataset = load_dataset(f"{hf_username}/pyomo-100")
train_test_split = dataset["train"].train_test_split(test_size=0.1, seed=42)

train_dataset = train_test_split["train"].map(generate_prompt_str)
eval_dataset = train_test_split["test"].map(generate_prompt_str)
test_dataset = dataset["test"].map(generate_test_prompt_str)

print(train_dataset)
print(eval_dataset)
print(test_dataset)

Dataset({
    features: ['input', 'context', 'output', 'text'],
    num_rows: 72
})
Dataset({
    features: ['input', 'context', 'output', 'text'],
    num_rows: 8
})
Dataset({
    features: ['input', 'context', 'output', 'text'],
    num_rows: 20
})


## Model

In [7]:
model_name = "microsoft/Phi-3-mini-128k-instruct"  # https://huggingface.co/microsoft/Phi-3-mini-128k-instruct
# model_name = "meta-llama/Llama-2-7b-chat-hf"
# model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
# model_name = "codellama/CodeLlama-7b-hf"

compute_dtype = getattr(torch, "float16")

# checkpoint_dir = "./checkpoints/" # First time training
# checkpoint_dir = "./merged_model/"  # Continue from merged model
checkpoint_dir = f"{hf_username}/Phi-3-mini-128k-instruct-pyomo"  # Fine-tuned model


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    checkpoint_dir,
    device_map=device,
    torch_dtype=compute_dtype,
    quantization_config=bnb_config,
    trust_remote_code=True,
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(
    checkpoint_dir,
    trust_remote_code=True,
)
tokenizer

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaTokenizerFast(name_or_path='wickes1/Phi-3-mini-128k-instruct-pyomo', vocab_size=32000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, 

## Training

In [18]:
peft_config = LoraConfig(
    bias="none",
    lora_alpha=16,
    lora_dropout=0.05,
    r=8,
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

training_args = TrainingArguments(
    bf16=False,
    evaluation_strategy="epoch",  # save checkpoint every epoch
    fp16=True,
    gradient_accumulation_steps=8,  # number of steps before performing a backward/update pass
    gradient_checkpointing=True,  # use gradient checkpointing to save memory
    group_by_length=True,
    learning_rate=2e-4,  # learning rate, based on QLoRA paper
    logging_steps=25,  # log every 10 steps
    lr_scheduler_type="cosine",  # use cosine learning rate scheduler
    max_grad_norm=0.3,  # max gradient norm based on QLoRA paper
    max_steps=-1,
    num_train_epochs=3,  # number of training epochs
    optim="paged_adamw_32bit",
    output_dir=checkpoint_dir,  # directory to save and repository id
    per_device_train_batch_size=1,  # batch size per device during training
    report_to="tensorboard",  # report metrics to tensorboard
    resume_from_checkpoint=True,
    save_steps=10,
    warmup_ratio=0.03,  # warmup ratio based on QLoRA paper
    weight_decay=0.001,
)


trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    max_seq_length=1024,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": True,
        # "append_concat_token": False,
    },
)

Map:   0%|          | 0/72 [00:00<?, ? examples/s]

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

In [19]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = AMD Radeon Graphics. Max memory = 15.984 GB.
5.072 GB of memory reserved.


In [20]:
# Train model
# trainer_stats = trainer.train()

# trainer.save_model("./checkpoints")
# tokenizer.save_pretrained("./checkpoints")
# trainer.save_metrics("metrics.json")

# tokenizer.push_to_hub(f"{hf_username}/Phi-3-mini-128k-instruct-pyomo", private=True)
# trainer.push_to_hub(f"{hf_username}/Phi-3-mini-128k-instruct-pyomo", private=True)
# trainer_stats.metrics

In [ ]:
# Merge PEFT model with the original model
# import gc

# import torch
# from peft import AutoPeftModelForCausalLM
# from transformers import AutoTokenizer

# for _ in range(100):
#     torch.cuda.empty_cache()
#     gc.collect()

# compute_dtype = getattr(torch, "float16")
# model_name = "microsoft/Phi-3-mini-128k-instruct"
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# tokenizer = AutoTokenizer.from_pretrained(
#     model_name,
# )

# model = AutoPeftModelForCausalLM.from_pretrained(
#     "./checkpoints/",  # PEFT Fine-tuned model
#     torch_dtype=compute_dtype,
#     return_dict=False,
#     low_cpu_mem_usage=True,
#     device_map=device,
#     trust_remote_code=True,
# )

# merged_model = model.merge_and_unload()
# merged_model.save_pretrained(
#     "./merged_model", safe_serialization=True, max_shard_size="2GB"
# )
# tokenizer.save_pretrained("./merged_model")

# merged_model.push_to_hub(f"{hf_username}/Phi-3-mini-128k-instruct-pyomo", private=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Testing

In [ ]:
def generate_answer(sample):
    test_prompt = generate_test_prompt_str(sample)["text"]
    model_input = tokenizer(test_prompt, return_tensors="pt").to(device)

    input_ids = model_input.input_ids.to(model.device)
    attention_mask = model_input.attention_mask.to(model.device)

    with torch.no_grad():
        gen_token = model.generate(
            input_ids, attention_mask=attention_mask, max_new_tokens=512
        )
    prompt_length = model_input.input_ids.shape[-1]
    gen_token_without_prompt = gen_token[:, prompt_length:]
    gen_text = tokenizer.decode(gen_token_without_prompt[0], skip_special_tokens=True)
    sample["answer"] = gen_text
    return sample


test_dataset = test_dataset.map(generate_answer, batched=False)

Parameter 'function'=<function generate_answer at 0x76147fbde660> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/20 [00:00<?, ? examples/s]

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


In [8]:
import random

from transformers import TextStreamer

model.eval()

test_sample = test_dataset[random.randint(0, len(test_dataset) - 1)]
gen_token = test_sample["output"]

test_prompt = generate_test_prompt_str(test_sample)["text"]
model_input = tokenizer(test_prompt, return_tensors="pt").to(device)

text_streamer = TextStreamer(tokenizer, skip_prompt=True)
gen_token = model.generate(
    model_input.input_ids,
    streamer=text_streamer,
    max_new_tokens=512,
)
prompt_length = model_input.input_ids.shape[-1]
gen_token_without_prompt = gen_token[:, prompt_length:]
gen_text = tokenizer.decode(gen_token_without_prompt[0], skip_special_tokens=True)
# print(gen_text)

# import subprocess
# import sys

# # execute the code
# result = subprocess.run([sys.executable, "-c", gen_text], capture_output=True)
# print(result.stdout.decode("utf-8"))
# print(result.stderr.decode("utf-8"))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical differences.


from pyomo.environ import *

model = ConcreteModel()
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)
model.obj = Objective(expr=2 * model.x + 3 * model.y, sense=maximize)

# Define constraints
model.constraint1 = Constraint(expr=model.x + 2 * model.y <= 8)
model.constraint2 = Constraint(expr=3 * model.x + 2 * model.y <= 12)
model.constraint3 = Constraint(expr=3 * model.x + 6 * model.y <= 50)

# Solve the model
solver = SolverFactory('glpk')
result = solver.solve(model)
<|end|>


In [ ]:
# Methods to generate text from the model

# M1
# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
# )

# generation_args = {
#     "max_new_tokens": 250,
#     "return_full_text": False,
#     "temperature": 0,
#     "do_sample": False,
# }

# for out in pipe(test_prompt, **generation_args):
#     print(out["generated_text"], end="", flush=True)

# M2
# with torch.no_grad():
#     print(tokenizer.decode(model.generate(**model_input, max_new_tokens=1000)[0], skip_special_tokens=True))

# M3
# outputs = model.generate(input_ids=model_input.input_ids, max_new_tokens=250)
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

# outputs = model.generate(input_ids=model_input.input_ids, max_new_tokens=500, do_sample=True, top_k=50, top_p=0.95)
# print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

# M4
# stop_token = "<|endoftext|>"
# stop_token_id = tokenizer.encode(stop_token)[0]
# model_input = tokenizer("hi", return_tensors="pt").to(device)

# _ = model.generate(
#     model_input.input_ids,
#     streamer=text_streamer,
#     max_new_tokens=256,
#     eos_token_id=stop_token_id,
#     do_sample=True,
# )

## Evaluation (Ragas)

In [ ]:
# Load PreTrained Model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

compute_dtype = getattr(torch, "float16")

pretrained_model = "microsoft/Phi-3-mini-128k-instruct"
finetuned_model = f"{hf_username}/Phi-3-mini-128k-instruct-pyomo"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model,
    trust_remote_code=True,
    device_map="cuda:0",
    quantization_config=bnb_config,
)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import copy


dataset = copy.deepcopy(test_dataset)
dataset = dataset.rename_columns(
    {"input": "question", "output": "ground_truth", "context": "contexts"}
)
dataset

# convert contexts to list
dataset = dataset.map(lambda x: {"contexts": [x["contexts"]]})
# dataset.to_pandas() # Quick check

In [ ]:
import importlib

import ragas.metrics
from dotenv import load_dotenv
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings
from ragas import evaluate
from ragas.metrics import (
    AspectCritique,
    answer_correctness,
    answer_relevancy,
    answer_similarity,
    context_precision,
    context_recall,
    faithfulness,
)
from ragas.metrics.critique import correctness, harmfulness
from ragas.run_config import RunConfig

importlib.reload(ragas.metrics)
load_dotenv()


AZURE_API_VERSION = "2024-02-01"
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")

# AZURE_MODEL= 'gpt-35-turbo-16k'
AZURE_MODEL = "gpt-4-32k"
azure_model = AzureChatOpenAI(
    azure_deployment=AZURE_MODEL,
    model=AZURE_MODEL,
    api_version=AZURE_API_VERSION,
    temperature=0,
    max_tokens=None,
    max_retries=2,
)

azure_embeddings = AzureOpenAIEmbeddings(
    # model="text-embedding-3-small",
    model="text-embedding-ada-002",
    api_version=AZURE_API_VERSION,
    tiktoken_enabled=False,
)


ollama_embeddings = OllamaEmbeddings(model="nomic-embed-text")

code_quality = AspectCritique(
    name="code-quality",
    definition="Is the submission a valid code that well-structured, efficient, and free from errors?",
    strictness=1,  # Number of self check
)


metrics = [
    # LLM metrics
    faithfulness,
    answer_relevancy,
    answer_similarity,
    correctness,
    code_quality,
    # RAG metrics
    # context_recall,
    # context_precision,
    # harmfulness,
]

result = evaluate(
    dataset,
    metrics=metrics,
    llm=azure_model,
    embeddings=ollama_embeddings,
    is_async=True,
    run_config=RunConfig(timeout=30, max_wait=30, max_retries=2, max_workers=4),
)
result

Evaluating:   0%|          | 0/100 [00:00<?, ?it/s]

{'faithfulness': 0.8494, 'answer_relevancy': 0.5697, 'answer_similarity': 0.9994, 'correctness': 0.9000, 'code-quality': 0.9000}

In [ ]:
# export mean value to json
FILE_NAME = "evaluation-finetuned"
# FILE_NAME = "evaluation-pretrained"
result.scores.to_pandas().mean().to_json(f"{FILE_NAME}.json", orient="index", indent=4)
result.to_pandas().to_csv(f"{FILE_NAME}.csv", index=False)

In [ ]:
from IPython.display import display, HTML

df = result.to_pandas()
sample = df[df["code-quality"] == 0]

display(HTML(sample.to_html(max_rows=10, max_cols=None)))

,question,contexts,ground_truth,text,answer,code-quality
16,add a constraint to ensure Q is at least 10.,"[from pyomo.environ import *\n\nmodel = ConcreteModel()\nmodel.Q = Var(domain=NonNegativeReals)\nmodel.obj = Objective(expr=2 * model.Q, sense=maximize)\n\n# Define constraints\nmodel.constraint1 = Constraint(expr=model.Q <= 8)\n\n# Solve the model\nsolver = SolverFactory('glpk')\nresult = solver.solve(model)]","from pyomo.environ import *\n\nmodel = ConcreteModel()\nmodel.Q = Var(domain=NonNegativeReals)\nmodel.obj = Objective(expr=2 * model.Q, sense=maximize)\n\n# Define constraints\nmodel.constraint1 = Constraint(expr=model.Q <= 8)\nmodel.constraint2 = Constraint(expr=model.Q >= 10)\n\n# Solve the model\nsolver = SolverFactory('glpk')\nresult = solver.solve(model)","<|system|>\nYou are a highly knowledgeable AI assistant specializing in Python programming and Pyomo models. Based on the user input question and the provided code snippet, your task is to generate the complete Python code needed to address the user's query.\n\nInstructions:\n1. Review the user input question carefully.\n2. Analyze the provided code snippet within the context given.\n3. Generate the full Python code required to answer the user's question, ensuring the code is functional and correctly formatted.\n\nNotes:\n- Ensure that the generated code is properly indented and follows Python best practices.\n- Include necessary import statements and any relevant comments to enhance code readability.\n- Verify that the code aligns with the user's specific requirements as described in the input and context.\n<|end|>\n<|user|>\nInput:\nadd a constraint to ensure Q is at least 10.\n\nContext:\nfrom pyomo.environ import *\n\nmodel = ConcreteModel()\nmodel.Q = Var(domain=NonNegativeReals)\nmodel.obj = Objective(expr=2 * model.Q, sense=maximize)\n\n# Define constraints\nmodel.constraint1 = Constraint(expr=model.Q <= 8)\n\n# Solve the model\nsolver = SolverFactory('glpk')\nresult = solver.solve(model)\n\nComplete Python Code:\n<|end|>\n<|assistant|>","from pyomo.environ import *\n\nmodel = ConcreteModel()\nmodel.Q = Var(domain=NonNegativeReals)\nmodel.obj = Objective(expr=2 * model.Q, sense=maximize)\n\n# Define constraints\nmodel.constraint1 = Constraint(expr=model.Q <= 8)\nmodel.constraint2 = Constraint(expr=model.Q >= 10)\n\n# Solve the model\nsolver = SolverFactory('glpk')\nresult = solver.solve(model)\n",0


In [ ]:
result.to_pandas()

,question,contexts,ground_truth,text,answer,faithfulness,answer_relevancy,answer_similarity,correctness,code-quality
0,add a new variable P and a constraint P + Y >= 8.,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.750000,0.530712,0.999089,1,1
1,modify the objective function to maximize 3X +...,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.777778,0.451872,1.000000,1,1
2,change the domain of Z to NonNegativeIntegers.,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.600000,0.479246,0.998503,1,1
3,add a parameter G with value 25 and use it in ...,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.818182,0.551395,1.000000,1,1
4,introduce a new constraint 5X + Y + Z <= 60.,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.700000,0.475206,1.000000,1,1
5,change the solver from 'scip' to 'couenne'.,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.500000,0.683909,1.000000,1,1
6,add a constraint to ensure P is at most 20.,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.666667,0.559796,1.000000,1,1
7,remove the constraint X + Y - Z >= 5.,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.875000,0.453050,0.998652,1,1
8,change the sense of the objective function to ...,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.833333,0.604279,1.000000,1,1
9,add a parameter H with value 14 and use it in ...,[from pyomo.environ import *\n\nmodel = Concre...,from pyomo.environ import *\n\nmodel = Concret...,<|system|>\nYou are a highly knowledgeable AI ...,from pyomo.environ import *\n\nmodel = Concret...,0.727273,0.495061,0.997958,1,1
